<center>
<h1 style="font-family:verdana">
 🏘 Sistema de diàleg basat en regles 🏘 <h1>








<p>   🎯 <b>Objectiu</b>: en aquesta pràctica aprendrem a crear un assistent senzill de compra d'habitatges. El sistema haurà d'identificar i mostrar les cases que coincideixen amb les preferències de l'usuari. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

<p> ✨ <b>Contingut</b>: en primer lloc, començarem amb un exemple senzill en què el sistema llançarà unes preguntes i respostes predefinides i l'usuari haurà d'escollir. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent.

---

<h2> Índex </h2>


1. [Fitxer JSON](#section-one)
  * [Exercici 1](#ex-one)
2. [Sistema de diàleg senzill](#section-two)
  * [Exercici 2](#ex-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Millorem el sistema de diàleg](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)
---


In [125]:
import nltk

from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import json
import sys
import random

<h1><a name="section-one"> 1. Fitxer JSON </a></h1>

En primer lloc, carregarem el fitxer JSON per analitzar-ne l'estructura.

In [126]:
# El nostre base de coneixement (plantilles de respostes)
with open('data_base.json') as f:
  data = json.load(f)

print('Start message: ', data['start_message'])
print('End message: ', data['end_message'])
# Decideix quina pregunta fa ==> detectar info relevant de la resposta(entitats) == Context
print('Questions: ', data['questions'])
# S'afegeix Context al json (és dinàmic)
print('House: ', data['houses'])


Start message:  Welcome to the House Buying Assistant! My name is Bothome, and I'm here to help you find your ideal house. You can type 'quite', and only 'quite', at any time to exit the program.
May I have your name? If no, just click the 'Enter' or 'Space' button.
End message:  Thank you for using the House Buying Assistant. Goodbye!
Questions:  [{'question': 'How many bedrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 5 bedrooms): ', 'invalid_response': "Invalid input. Please try again or write 'quite' to exit.", 'answer_key': 'bedrooms'}, {'question': 'How many bathrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 3 bathrooms): ', 'invalid_response': "Invalid input. Please try again or write 'quite' to exit.", 'answer_key': 'bathrooms'}, {'question': 'What is your budget for the house?', 'type': 'numerical', 'prompt': 'Enter your choice (1k - 999k euros):', 'invalid_response': "Invalid input. Please try again or write 'quite' to

Podeu veure com al fitxer JSON podem trobar el missatge de benvinguda i el missatge de comiat. Les diferents preguntes predefinides que el sistema demanarà a l'usuari. I les cases que l'agència té disponibles per a oferir a l'usuari.

Si no acabeu d'entendre el format, podeu copiar el contingut del fitxer JSON [aquí](https://jsonviewer.stack.hu/) per visualitzar millor l'estructura.

---



 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>


En aquest primer exercici us demanem que afegiu una nova casa manualment al fitxer JSON amb l'id 26. La resta de camps els podeu emplenar com vulgueu.

Si ho heu fet correctament podreu veure al següent *print* les dades que heu introduït.

In [127]:
'''id = 26
house = data['houses'][id-1]

print(f"House ID: {house['id']}")
print(f"Bedrooms: {house['bedrooms']}")
print(f"Bathrooms: {house['bathrooms']}")
print(f"Price: {house['price']}")
print(f"Square Meters: {house['square_meters']}")
print(f"Location: {house['location']}")'''

'id = 26\nhouse = data[\'houses\'][id-1]\n\nprint(f"House ID: {house[\'id\']}")\nprint(f"Bedrooms: {house[\'bedrooms\']}")\nprint(f"Bathrooms: {house[\'bathrooms\']}")\nprint(f"Price: {house[\'price\']}")\nprint(f"Square Meters: {house[\'square_meters\']}")\nprint(f"Location: {house[\'location\']}")'

---

<h1><a name="section-two"> 2. Sistema de diàleg senzill </a></h1>


En aquesta primera part crearem un sistema senzill que mostrarà les preguntes predefinides i, a continuació l'usuari haurà d'introduir la resposta. Tractarem de manera diferent les preguntes en què la resposta siga un número i les preguntes d'opció múltiple.

Intenteu entendre el funcionament, ja que treballarem sobre aquest codi.

In [128]:
# Print question
def print_question(prompt, possible_options = []):
    print(prompt)
    if not len(possible_options) == 0:
      print("Options:", ", ".join(possible_options))

# Omplint la base de knowledge
def initialize_available_options(house_data, available_options):
    for house in house_data['houses']:
        for key, value in house.items():
            available_options.setdefault(key, set()).add(value)

def preprocess_answer(answer):
    answer = nltk.word_tokenize(answer)
    return answer

# Detectar entitats
def get_numerical_value(tok_answer):
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric():
            return token
    return ''

#Exemple de frase tokenitzada
print(preprocess_answer('Vull un pis de 35k €'))

['Vull', 'un', 'pis', 'de', '35k', '€']


In [129]:
'''# 2 tipus de preguntas (resposta numèriques y textuals)
def process_numerical_question(question):
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      if answer in options:
        return answer

user_preferences, available_options = {}, {}
initialize_available_options(data, available_options)
print(available_options)

################ BOT ###################

for question in data['questions']:
  answer_key = question['answer_key']
  print(answer_key)
  possible_options = list(available_options.get(answer_key))
  

  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer
'''
# Un cop acabat, és necessari provar-ho amb camins feliços y no feliços.'''

"# 2 tipus de preguntas (resposta numèriques y textuals)\ndef process_numerical_question(question):\n    print_question(question['question'])\n    while True:\n      answer = input(question['prompt'])\n      tok_answer = preprocess_answer(answer)\n      value = get_numerical_value(tok_answer)\n      if not value == '':\n        return value\n\ndef process_multichoice_question(question, options):\n    print_question(question['question'], options)\n    while True:\n      answer = input(question['prompt'])\n      if answer in options:\n        return answer\n\nuser_preferences, available_options = {}, {}\ninitialize_available_options(data, available_options)\nprint(available_options)\n\n################ BOT ###################\n\nfor question in data['questions']:\n  answer_key = question['answer_key']\n  print(answer_key)\n  possible_options = list(available_options.get(answer_key))\n  \n\n  if question['type'] == 'numerical':\n    answer = process_numerical_question(question)\n  else:\n



---


 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏ </a></h1>

A continuació millorarem un poc el sistema. Us proposem que afegiu la frase de benvinguda que tenim guardada al fitxer JSON. És a dir, abans de mostrar la primera pregunta el sistema ens donarà la benvinguda.

---


 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏ </a></h1>

En aquest exercici us demanem que permeteu a l'usuari abandonar el programa quan ho desitge. Per exemple, si l'usuari escriu `quit`, que el sistema s'acomiade amb l'oració de comiat que trobareu al JSON i finalitze el programa.


🙃 `sys.exit()` genera una excepció SystemExit.

Funciona també si escriviu `Quit`?

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏ </a></h1>

Fins ara tenim a `user_preferences` les opcions que l'usuari ha escollit.







In [130]:
'''user_preferences'''

'user_preferences'

Per tant, ara haurem de revisar si l'agència disposa d'alguna casa disponible amb aquestes característiques. Per això us proposem completar la funció `find_suitable_houses`. Aquesta funció tornarà una llista amb tants diccionaris com cases disponibles. Per exemple si segons les preferències de l'usuari només està disponible la casa amb id 3, `suitable_house` serà:


```
[{'id': 3, 'bedrooms': '2', 'bathrooms': '1', 'price': '160k', 'square_meters': '80', 'location': 'Santa Coloma de Gramenet'}]
```

In [131]:
#TODO
#def find_suitable_houses(data, user_preferences):
    #...
    #return suitable_houses

---



Una vegada tingueu la funció `find_suitable_houses` programada, podreu executar el següent *script* que us mostrarà, si n'hi ha, la casa o cases disponibles amb les característiques que heu triat.

In [132]:
'''def print_suitable_houses(suitable_houses):
  if suitable_houses:
    print("\nBased on your preferences, the most suitable houses are:")
    for house in suitable_houses:
      print(f"House ID: {house['id']}")
      print(f"Type: {house['type']}")
      print("Bedrooms:", house['bedrooms'])
      print("Bathrooms:", house['bathrooms'])
      print("Price:", house['price'], "euros")
      print("Square Meters:", house['square_meters'], "m^2")
      print("Location:", house['location'])
      print()
  else:
    print("\nSorry, no suitable houses match your preferences. \n")


suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)'''

'def print_suitable_houses(suitable_houses):\n  if suitable_houses:\n    print("\nBased on your preferences, the most suitable houses are:")\n    for house in suitable_houses:\n      print(f"House ID: {house[\'id\']}")\n      print(f"Type: {house[\'type\']}")\n      print("Bedrooms:", house[\'bedrooms\'])\n      print("Bathrooms:", house[\'bathrooms\'])\n      print("Price:", house[\'price\'], "euros")\n      print("Square Meters:", house[\'square_meters\'], "m^2")\n      print("Location:", house[\'location\'])\n      print()\n  else:\n    print("\nSorry, no suitable houses match your preferences. \n")\n\n\nsuitable_houses = find_suitable_houses(data, user_preferences)\nprint_suitable_houses(suitable_houses)'

<h1><a name="section-three"> 3. Millorem el sistema de diàleg </a></h1>


---


 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Tal com haureu comprovat sempre esteu obligats a triar una opció. En aquest últim exercici proposem que afegiu l'opció de no triar-ne cap. És a dir, si ens és igual el nombre d'habitacions, podrem posar per exemple `any` i el sistema ens farà la següent pregunta.

In [133]:
'''# Complete Exercici 5 implementation with improved features
user_preferences, available_options = {}, {}
initialize_available_options(data, available_options)
print(available_options)

def process_numerical_question(question):
    print_question(question['question'])
    while True:
        answer = input(question['prompt'])
        
        if answer.lower() == 'quite':
            return 'quite'
        
        if answer.strip() == '' or answer.lower() == 'any':
            return 'any'
        
        tok_answer = preprocess_answer(answer)
        value = get_numerical_value(tok_answer)
        
        if not value == '':
            return value
        
        print(question['invalid_response'])

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
        answer = input(question['prompt'])
        
        if answer.lower() == 'quite':
            return 'quite'
        
        if answer.strip() == '' or answer.lower() == 'any':
            return 'any'
        
        if answer in options:
            return answer
        
        print(question['invalid_response'])

def chatbox_withANYoption():
    # Get user name
    user_name = get_user_name()
    
    for question in data['questions']:
        answer_key = question['answer_key']
        
        # Skip income question if user chose 'sale' or 'any' for house type
        if answer_key == 'income':
            if user_preferences.get('type') == 'sale' or user_preferences.get('type') == 'any':
                user_preferences['income'] = 'any'
                continue
        
        if answer_key == 'income':
          possible_options = []
        else:
          possible_options = list(available_options.get(answer_key))

        if question['type'] == 'numerical':
            answer = process_numerical_question(question)
        else:
            answer = process_multichoice_question(question, possible_options)

        user_preferences[answer_key] = answer

chatbox_withANYoption()'''

"# Complete Exercici 5 implementation with improved features\nuser_preferences, available_options = {}, {}\ninitialize_available_options(data, available_options)\nprint(available_options)\n\ndef process_numerical_question(question):\n    print_question(question['question'])\n    while True:\n        answer = input(question['prompt'])\n        \n        if answer.lower() == 'quite':\n            return 'quite'\n        \n        if answer.strip() == '' or answer.lower() == 'any':\n            return 'any'\n        \n        tok_answer = preprocess_answer(answer)\n        value = get_numerical_value(tok_answer)\n        \n        if not value == '':\n            return value\n        \n        print(question['invalid_response'])\n\ndef process_multichoice_question(question, options):\n    print_question(question['question'], options)\n    while True:\n        answer = input(question['prompt'])\n        \n        if answer.lower() == 'quite':\n            return 'quite'\n        \n       

In [134]:
'''print(user_preferences)'''

'print(user_preferences)'

---



 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Amplieu el vostre xatbot perquè tinga en compte la següent informació sobre:

1. **Tipus d'habitatge:** Pregunteu a l'usuari si vol una casa per a compra (tipus *"sale"* a les dades) o per llogar (tipus *"rent"*). Al final de l'execució, el sistema haurá de mostrar habitatges amb el tipus seleccionat.

2. **Ingressos:** S'aconsella no dedicar més del 35% dels ingressos a l'habitatge. En cas d'escollir lloguer, preguntar a l'usuari els ingressos mensuals de la seua llar, i només mostrar aquells que estiguen per sota del 35%.

3. **Planta:** Preguntar als usuaris a quina planta volen viure. Al final de l'execució, només s'oferiràn habitatges que estiguen en aquella planta o superior.

4. **Terrassa:** Preguntar a l'usuari si vol un habitatge amb terrassa o no. En cas de voler terrassa, només s'oferiran habitatges amb terrassa. En cas contrari, oferir habitatges amb terrassa o sense.

5. **Ascensor:** Preguntar a l'usuari si vol un habitatge amb ascensor o no. En cas de voler ascensor, només s'oferiran habitatges amb ascensor. En cas contrari, oferir habitatges amb ascensor o sense.

6. **Us comercial:** Preguntar a l'usuari si vol fer servir l'habitatge com a negoci. En cas afirmatiu, només oferir habitatges amb la propietat *"commercial use"*


EXTRA: AÑADIR CONECTORES ==> PERSONALIDAD BOT 
---





In [135]:
#TO DO

#TODO
def find_suitable_houses(data, user_preferences):
    def extract_number(text):
        digits = ''.join(filter(str.isdigit, str(text)))
        if digits and text[-1] in ['K', 'k']:
            return int(digits) * 1000
        return int(digits) if digits else text

    def mayor_mejor(house_value, pref_value):
        return house_value >= pref_value
    def igual_mejor (house_value, pref_value):
        return house_value == pref_value
    def menor_mejor (house_value, pref_value):
        return house_value <= pref_value

    # Para tratar las características binarias que son más restrictivos con 'Yes', se podría implementar poniendo un 'any'
    def binario(house_value: str, pref_value: str) -> bool:  
        if pref_value.lower() in ("yes", "y", "true", "1"):
            return house_value == 'Yes'
        else:
            return True

    # CASO EXTREMO SI EL USUARIO NO TIENE NINGÚN CRITERIO.
    all_any = all(value == 'any' for value in user_preferences.values())
    if all_any:
        return data['houses']
    
    criterio = {
        'mayor_mejor': {'bedrooms', 'bathrooms', 'square_meters', 'floor'}, 
        'igual_mejor': {'location', 'type'}, 
        'menor_mejor': {'price'}, 
        'preguntas_binarias': {'terrace', 'elevator', 'commercial_use'}
    }
    
    comparacion = {
        'mayor_mejor': mayor_mejor, 
        'igual_mejor': igual_mejor, 
        'menor_mejor': menor_mejor, 
        'preguntas_binarias': binario
    }
    
    suitable_houses = []
    for house in data['houses']:
        match = True
        for key, value in user_preferences.items():
            if value == 'any':
                continue
            if key == 'income' and value != 'any':
                if house['type'] == 'rent':
                    income = extract_number(value)
                    price = extract_number(house['price'])
                    if price > income * 0.35:  # 35% rule
                        match = False
                        break
                continue
            
            # Regular criteria matching
            for crit, keys in criterio.items():
                if key in keys:
                    house_value = extract_number(house[key])
                    pref_value = extract_number(value)
                    if not comparacion[crit](house_value, pref_value):
                        match = False
                        break
            if not match:                
                break          
        if match:
            suitable_houses.append(house)
    return suitable_houses

In [136]:
'''suitable_houses = find_suitable_houses(data, user_preferences)

# Display results with personalized message if name was provided
if 'user_name' in locals() and user_name:
    print(f"\n{user_name}, here are the suitable houses based on your preferences:")
else:
    print("\nSuitable houses based on your preferences:")

if suitable_houses:
    for house in suitable_houses:
        print(f"House ID: {house['id']}, Type: {house['type']}, Bedrooms: {house['bedrooms']}, Bathrooms: {house['bathrooms']}, Price: {house['price']}, Square Meters: {house['square_meters']}, Location: {house['location']}")
else:
    print(data['no_suitable_houses_message'])

# Personalized goodbye
if 'user_name' in locals() and user_name:
    print(f"\nThank you for using our service, {user_name}! Have a great day!")'''

'suitable_houses = find_suitable_houses(data, user_preferences)\n\n# Display results with personalized message if name was provided\nif \'user_name\' in locals() and user_name:\n    print(f"\n{user_name}, here are the suitable houses based on your preferences:")\nelse:\n    print("\nSuitable houses based on your preferences:")\n\nif suitable_houses:\n    for house in suitable_houses:\n        print(f"House ID: {house[\'id\']}, Type: {house[\'type\']}, Bedrooms: {house[\'bedrooms\']}, Bathrooms: {house[\'bathrooms\']}, Price: {house[\'price\']}, Square Meters: {house[\'square_meters\']}, Location: {house[\'location\']}")\nelse:\n    print(data[\'no_suitable_houses_message\'])\n\n# Personalized goodbye\nif \'user_name\' in locals() and user_name:\n    print(f"\nThank you for using our service, {user_name}! Have a great day!")'

 <h1><a name="ex-seven"><center> ✏ Exercici 7 ✏ </a></h1>

Ara ja teniu un sistema funcional. Aplicant el mètode vist a classe (Exemples, Camins, Prototipat i Proves) milloreu aquest sistema per donar una millor experiència als usuaris.    


In [ ]:

# Exercici 3 - Complete implementation with name support
def get_user_name():
    """Get user's name and return personalized greeting"""
    print(data['start_message'])
    name = input().strip()
    
    if name.lower() == 'quite':
        print(data['end_message'])
        return 'quite'
    
    if name == '' or name == ' ':
        return None
    
    # Random personalized greeting
    if name:
        greeting = random.choice(data['name_prompts']).format(name=name)
        print(greeting)
    
    return name

def process_numerical_question_with_name(question, user_name=None):
    """Process numerical questions with optional personalized messages"""
    print_question(question['question'])
    
    while True:
        answer = input(question['prompt'])
        
        if answer.lower() == 'quite':
            return 'quite'
        
        if answer.strip() == '' or answer.lower() == 'any':
            # 30% chance to use personalized message if user has name
            if user_name and random.random() < 0.3:
                print(random.choice(data['personalized_messages']).format(name=user_name))
            return 'any'
        
        tok_answer = preprocess_answer(answer)
        value = get_numerical_value(tok_answer)
        
        if not value == '':
            # 40% chance to use personalized message if user has name
            if user_name and random.random() < 0.4:
                print(random.choice(data['personalized_messages']).format(name=user_name))
            return value
        
        print(question['invalid_response'])

def process_multichoice_question_with_name(question, options, user_name=None):
    """Process multichoice questions with optional personalized messages"""
    print_question(question['question'], options)
    
    while True:
        answer = input(question['prompt'])
        
        if answer.lower() == 'quite':
            return 'quite'
        
        if answer.strip() == '' or answer.lower() == 'any':
            # 30% chance to use personalized message if user has name
            if user_name and random.random() < 0.5:
                print(random.choice(data['personalized_messages']).format(name=user_name))
            return 'any'
        
        if answer in options:
            # 40% chance to use personalized message if user has name
            if user_name and random.random() < 0.4:
                print(random.choice(data['personalized_messages']).format(name=user_name))
            return answer
        
        print(question['invalid_response'])



# Test the enhanced system
def test_enhanced_system():
    """Complete chatbot system with name memory and personalization"""
    
    def complete_chatbot():
        user_preferences, available_options = {}, {}
        initialize_available_options(data, available_options)
        
        # Get user name with welcome message
        user_name = get_user_name()

        if user_name == 'quite':
            print(data['end_message'])
            return

        
        # Process all questions with personalization
        for question in data['questions']:
            answer_key = question['answer_key']
            
            # Skip income question if not renting
            if answer_key == 'income':
                if user_preferences.get('type') in ['sale', 'any']:
                    user_preferences['income'] = 'any'
                    continue
            
            if answer_key == 'income':
                possible_options = []
            else:
                possible_options = list(available_options.get(answer_key))

            # Ask question with personalization
            if question['type'] == 'numerical':
                answer = process_numerical_question_with_name(question, user_name)
            else:
                answer = process_multichoice_question_with_name(question, possible_options, user_name)

            if answer == 'quite':
                if user_name:
                    print(f"Goodbye {user_name}! {data['end_message']}")
                else:
                    print(data['end_message'])
                return

            user_preferences[answer_key] = answer

        # Find and display suitable houses
        suitable_houses = find_suitable_houses(data, user_preferences)
        
        if suitable_houses:
            if user_name:
                print(f"\n{user_name}, based on your preferences, here are the most suitable houses:")
            else:
                print("\nBased on your preferences, here are the most suitable houses:")
            
            for house in suitable_houses:
                print(f"🏠 House ID: {house['id']}")
                print(f"   Type: {house['type']}")
                print(f"   Bedrooms: {house['bedrooms']}, Bathrooms: {house['bathrooms']}")
                print(f"   Price: {house['price']}, Size: {house['square_meters']} m²")
                print(f"   Location: {house['location']}")
                print(f"   Floor: {house['floor']}, Elevator: {house['elevator']}")
                print(f"   Terrace: {house['terrace']}, Commercial use: {house['commercial_use']}")
                print()
        else:
            if user_name:
                print(f"Sorry {user_name}, {data['no_suitable_houses_message']}")
            else:
                print(data['no_suitable_houses_message'])
        
        # Final personalized goodbye
        if user_name:
            print(f"Thank you for using our service, {user_name}! I hope we helped you find your perfect home. {data['end_message']}")
        else:
            print(data['end_message'])
    
    # Run the complete chatbot
    complete_chatbot()

# Uncomment to test the enhanced system
test_enhanced_system()

Welcome to the House Buying Assistant! My name is Bothome, and I'm here to help you find your ideal house. You can type 'quite', and only 'quite', at any time to exit the program.
May I have your name? If no, just click the 'Enter' or 'Space' button.
How many bedrooms do you need?
How many bathrooms do you need?
What is your budget for the house?
How many square meters do you need?
Which city or neighborhood would you prefer?
Options: Esplugues de Llobregat, Santa Coloma de Gramenet, Barcelona, L'Hospitalet de Llobregat
Do you want a house for buying or renting?
Options: sale, rent
From which floor would you like to live?
Do you want the house to have a terrace?
Options: Yes, No
Do you want the house to have an elevator?
Options: Yes, No
Do you need the house to be suitable for commercial use?
Options: Yes, No

Based on your preferences, here are the most suitable houses:
🏠 House ID: 1
   Type: rent
   Bedrooms: 3, Bathrooms: 2
   Price: 1200, Size: 100 m²
   Location: Santa Coloma de 

<h1><a name="section-four"> 4. Lliurable </a></h1>

 *CASOS LÍMITS (Ctrl C, quit)*

*Comprovar a cada pregunta si queda alguna opció disponible.*

*Més flexibilitat en la resposta (NER per detectar la resposta o detecció de sentiments)*

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Resultats (amb raonament).
3. Conversation flow del vostre sistema final. (PPT)
4. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.


---